In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [16]:
!wget $data -O data.csv

--2025-10-15 17:40:07--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  78.98K  --.-KB/s    in 0.001s  

2025-10-15 17:40:07 (90.5 MB/s) - ‘data.csv’ saved [80876/80876]



In [17]:
df = pd.read_csv(data)

In [18]:
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


#### Data Preparation

In [19]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [20]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [21]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [22]:
df[['lead_source','industry', 'employment_status', 'location']] = df[['lead_source','industry', 'employment_status', 'location']].fillna('NA')

In [23]:
df['annual_income'] = df.annual_income.fillna(0.0)

Q1

In [24]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

Q2 : Calculate correlation matrix

In [25]:
corr = df.select_dtypes(include=['number']).corr()
corr

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879,0.435914
annual_income,0.009770,1.000000,0.027036,0.015610,0.053131
interaction_count,-0.023565,0.027036,1.000000,0.009888,0.374573
lead_score,-0.004879,0.015610,0.009888,1.000000,0.193673
converted,0.435914,0.053131,0.374573,0.193673,1.000000


#### Split the data

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [28]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [29]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [30]:
df_full_train = df_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [31]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [32]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

Q3

In [44]:
from sklearn.metrics import mutual_info_score

In [40]:
categorical = ['lead_source', 'industry', 
       'employment_status', 'location',
       ]

In [71]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

In [48]:
def mutual_info(series):
    return mutual_info_score(series, df_full_train.converted)

In [49]:
mi = df_full_train[categorical].apply(mutual_info)
mi.sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

In [64]:
from sklearn.feature_extraction import DictVectorizer

In [65]:
df[categorical].nunique()

lead_source          6
industry             8
employment_status    5
location             8
dtype: int64

In [72]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [73]:
dv = DictVectorizer(sparse=False)

In [74]:
X_train = dv.fit_transform(train_dicts)

In [75]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [76]:
X_val = dv.transform(val_dicts)

In [78]:
from sklearn.linear_model import LogisticRegression

In [80]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [82]:
y_pred = model.predict_proba(X_val)

In [83]:
df_val[(y_pred >= 0.5)].astype(int)

ValueError: invalid literal for int() with base 10: 'paid_ads'